In [2]:
import pickle
import numpy as np
import pandas as pd
import math
from NLTK_cleaning import clean 

def tf(term, text):
    term_count = text.count(term)
    total_words = len(text)
    return term_count / total_words  

def idf(term, df):
    count = df["cleaned description"].apply(lambda x: term in x if isinstance(x, str) else False).sum()
    print(count)
    return math.log((len(df))/(count))        


def calculate_cosine_similarity(query_tfidf, document_tfidf):
    dot_product = np.dot(query_tfidf, document_tfidf)
    norm_query = np.linalg.norm(query_tfidf)
    norm_document = np.linalg.norm(document_tfidf)
    if norm_query == 0 or norm_document == 0:
        return 0.0 
    cosine_similarity = dot_product / (norm_query * norm_document)
    return cosine_similarity

with open(r"C:\Users\youse\Desktop\GitHub\adm-hw3\Databases\saved_dictionary_tfidf.pkl", 'rb') as f:
    tfidf_dict = pickle.load(f)
with open(r"C:\Users\youse\Desktop\GitHub\adm-hw3\Databases\saved_dictionary.pkl", 'rb') as f:
    inverted_index = pickle.load(f)

vocab_path = r'C:\Users\youse\Desktop\GitHub\adm-hw3\Databases\Vocabulary.csv'
vocab = pd.read_csv(vocab_path)

degrees = pd.read_csv(r"C:\Users\youse\Desktop\GitHub\adm-hw3\Databases\Parsed_database.csv")
degrees = degrees.drop(columns=["Unnamed: 0"])





In [34]:
query = "big data engineering "

match_list = []
query = clean(query).lower().split(" ")

for lemma in query:
    word_id = vocab[vocab['term'] == lemma]['term_id']
    print(word_id)
    try:
        word_id = word_id.values[0]
        matches = set(inverted_index[word_id]) 
        match_list.append(matches)
    except:
        print(f"{lemma} not in dataset. It will not be included in search.")
        query.remove(lemma)

result_id = set.intersection(*match_list)   
all_results = pd.DataFrame(columns=["courseName", "universityName", "description", "url"])

query_tfidf={}
tfidf_query_corpus = {}
for word in query:
    tf_query = tf(word, query)
    idf_query = idf(word, degrees)
    query_tfidf[word] = tf_query*idf_query


for result in result_id:
    doc = []
    for word in query:
        word_id = vocab[vocab['term'] == word]['term_id'].values[0]
        doc.append([x[1] for x in tfidf_dict[word_id] if x[0]== result][0])
    tfidf_query_corpus[result] = doc
    
print(query_tfidf)
print(tfidf_query_corpus)      
    
        


    





203    100203
Name: term_id, dtype: int64
204    100204
Name: term_id, dtype: int64
91    100091
Name: term_id, dtype: int64
217
778
780
{4371, 5787, 29, 1314, 1315, 681, 4783, 4784, 50, 5690, 3389, 65, 5190, 5585, 1234, 5587, 3289, 1504, 3686, 4582, 1259, 1145, 5887}
{'big': 1.1065391315565773, 'data': 0.6809294080106001, 'engineering': 0.6800736095088515}
{4371: [0.09471960966286926, 0.05833343871767899, 0.05826017860428077], 5787: [0.04196438402785346, 0.025843928545807145, 0.025811471533542112], 29: [0.06255068562642309, 0.038522082172052156, 0.03847370285188353], 1314: [0.06139233959630414, 0.03780871027997712, 0.03776122687314494], 1315: [0.08500477790257496, 0.05235052192612216, 0.052284775670508384], 681: [0.07534514405000964, 0.04640159897997192, 0.0463433238897688], 4783: [0.07053587953617924, 0.0868795895795219, 0.04338523938616653], 4784: [0.07053587953617924, 0.0868795895795219, 0.04338523938616653], 50: [0.15069028810001928, 0.09280319795994384, 0.0926866477795376], 5690: